# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
#!pip install pandas

## LLMChain

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [7]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
product = "Queen Size Sheet Set"
resp = chain.invoke({"product":product})

In [10]:
resp['text']

'Royal Bedding Co.'

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True)

In [15]:
overall_simple_chain.run(product)

/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
Royal Linens Co.
Royal Linens Co. provides high-quality linens for residential and commercial spaces with a wide range of fabrics and colors to choose from.

> Finished chain.


'Royal Linens Co. provides high-quality linens for residential and commercial spaces with a wide range of fabrics and colors to choose from.'

## SequentialChain

In [16]:
from langchain.chains import SequentialChain

In [17]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [18]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [19]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [20]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [21]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [22]:
review = """malgré que mon matelas ne soit pas tant épais. Mais fait quand-même avec un peu de force."""

In [23]:
review = """These fit my queen bed perfectly! The sheets actually go under the mattress so they aren’t sliding off during the night so no more sheet clips. They are soft as well and not scratchy like I was afraid I’d get!"""

In [24]:
review

'These fit my queen bed perfectly! The sheets actually go under the mattress so they aren’t sliding off during the night so no more sheet clips. They are soft as well and not scratchy like I was afraid I’d get!'

In [25]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'These fit my queen bed perfectly! The sheets actually go under the mattress so they aren’t sliding off during the night so no more sheet clips. They are soft as well and not scratchy like I was afraid I’d get!',
 'English_Review': 'Estos se ajustan perfectamente a mi cama Queen. Las sábanas realmente quedan debajo del colchón, por lo que no se deslizan durante la noche, por lo que no necesito más clips para sábanas. También son suaves y no son rasposas como temía que fueran.',
 'summary': 'The sheets fit perfectly on the Queen bed, stay in place, and are soft and comfortable to sleep on.',
 'followup_message': "Thank you for sharing your positive experience with the Queen bed sheets. It's great to hear that they fit perfectly and stay in place while also being soft and comfortable for a good night's sleep. We appreciate your feedback and hope you continue to enjoy using them."}

## Router Chain

In [26]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [27]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [29]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [30]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [31]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [32]:
from langchain_core.output_parsers import JsonOutputParser

In [33]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [34]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [35]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

In [36]:
router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics: Good for answering questions about physics\nmath: Good for answering math q

In [37]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [38]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [39]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [40]:
chain.invoke("What are black holes?")



> Entering new MultiPromptChain chain...


/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is the theory behind black holes and how do they form?'}
> Finished chain.


{'input': 'What is the theory behind black holes and how do they form?',
 'text': 'Black holes are formed when a massive star runs out of fuel and collapses under its own gravity. This collapse causes the star to become incredibly dense, with all of its mass compressed into a single point known as a singularity. The gravitational pull of this singularity is so strong that nothing, not even light, can escape its grasp. This is why black holes are called "black" - they do not emit any light that we can see.\n\nThe theory behind black holes is based on Einstein\'s theory of general relativity, which describes how gravity works. According to this theory, massive objects like stars warp the fabric of space-time around them, causing other objects to be pulled towards them. When a star collapses into a black hole, it creates a region of space-time where the gravitational pull is so strong that it warps space-time to the point where it becomes infinitely curved. This is what creates the singul

In [41]:
chain.run("what is 2 * 2 + 2")



> Entering new MultiPromptChain chain...


/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'what is the result of 2 * 2 + 2?'}
> Finished chain.


'The result of 2 * 2 is 4. Adding 2 to 4 gives us a final answer of 6. Therefore, the result of 2 * 2 + 2 is 6.'

In [42]:
chain.run("What is the debate between nature and nurture?")



> Entering new MultiPromptChain chain...


/opt/homebrew/Caskroom/miniforge/base/envs/langchain/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


History: {'input': 'What is the debate between nature and nurture?'}
> Finished chain.


'The debate between nature and nurture is a long-standing argument in the field of psychology and sociology. It concerns the extent to which human behavior is determined by genetic factors (nature) or environmental factors (nurture). Some argue that our behavior is primarily determined by our genetic makeup, while others believe that our environment and experiences play a greater role in shaping who we are. The debate has important implications for understanding human development, mental health, and social policy. While there is no clear consensus on the issue, most experts agree that both nature and nurture play a role in shaping human behavior.'